In [2]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()
from functools import reduce
from django.core.serializers import serialize

In [3]:
from tra.models import *

In [4]:
from tra.storage import UTECloudLogsStorage

In [5]:
storage = UTECloudLogsStorage()

In [5]:
storage.save("2175", "http://logs.ute.nsn-rdnet.net/cloud/execution/15632187/")

--2022-08-24 15:35:33--  http://logs.ute.nsn-rdnet.net/cloud/execution/15632187/
Resolving lab-proxy.krk-lab.nsn-rdnet.net (lab-proxy.krk-lab.nsn-rdnet.net)... 10.158.100.2
Connecting to lab-proxy.krk-lab.nsn-rdnet.net (lab-proxy.krk-lab.nsn-rdnet.net)|10.158.100.2|:8080... connected.
Proxy request sent, awaiting response... 200 OK
Length: 484 [text/html]
Saving to: ‘index.html.tmp’

     0K                                                       100% 69.9M=0s

2022-08-24 15:35:34 (69.9 MB/s) - ‘index.html.tmp’ saved [484/484]

Loading robots.txt; please ignore errors.
--2022-08-24 15:35:35--  http://logs.ute.nsn-rdnet.net/robots.txt
Reusing existing connection to lab-proxy.krk-lab.nsn-rdnet.net:8080.
Proxy request sent, awaiting response... 404 Not Found
2022-08-24 15:35:35 ERROR 404: Not Found.

Removing index.html.tmp since it should be rejected.

--2022-08-24 15:35:35--  http://logs.ute.nsn-rdnet.net/cloud/execution/15632187/execution_for_preparation_2e17f041-3457-4ff4-ab71-871c73366

Removing execution_for_preparation_2e17f041-3457-4ff4-ab71-871c73366439/reporting_portal_report/collected/index.html.tmp since it should be rejected.

--2022-08-24 15:35:50--  http://logs.ute.nsn-rdnet.net/cloud/execution/15632187/execution_for_preparation_2e17f041-3457-4ff4-ab71-871c73366439/reporting_portal_report/reporting_portal.zip
Reusing existing connection to lab-proxy.krk-lab.nsn-rdnet.net:8080.
Proxy request sent, awaiting response... 200 OK
Length: 3035 (3.0K) [application/zip]
Saving to: ‘execution_for_preparation_2e17f041-3457-4ff4-ab71-871c73366439/reporting_portal_report/reporting_portal.zip’

     0K ..                                                    100%  333M=0s

2022-08-24 15:35:50 (333 MB/s) - ‘execution_for_preparation_2e17f041-3457-4ff4-ab71-871c73366439/reporting_portal_report/reporting_portal.zip’ saved [3035/3035]

--2022-08-24 15:35:51--  http://logs.ute.nsn-rdnet.net/cloud/execution/15632187/execution_for_preparation_2e17f041-3457-4ff4-ab71-871c73366439/re

   900K .......... .......... .......... .......... .......... 17% 1.52M 1s
   950K .......... .......... .......... .......... .......... 18% 1.72M 1s
  1000K .......... .......... .......... .......... .......... 19% 1.28M 1s
  1050K .......... .......... .......... .......... .......... 20% 2.11M 1s
  1100K .......... .......... .......... .......... .......... 21% 2.24M 1s
  1150K .......... .......... .......... .......... .......... 22% 1.27M 1s
  1200K .......... .......... .......... .......... .......... 23% 1.55M 1s
  1250K .......... .......... .......... .......... .......... 24% 1.80M 1s
  1300K .......... .......... .......... .......... .......... 25% 1.85M 1s
  1350K .......... .......... .......... .......... .......... 26% 1.64M 1s
  1400K .......... .......... .......... .......... .......... 27% 1.54M 1s
  1450K .......... .......... .......... .......... .......... 28% 1.54M 1s
  1500K .......... .......... .......... .......... .......... 29% 2.31M 1s
  1550K ....

   850K .......... .......... .......... .......... .......... 56% 1.81M 0s
   900K .......... .......... .......... .......... .......... 59% 1.61M 0s
   950K .......... .......... .......... .......... .......... 63% 1.92M 0s
  1000K .......... .......... .......... .......... .......... 66% 1.75M 0s
  1050K .......... .......... .......... .......... .......... 69% 1.12M 0s
  1100K .......... .......... .......... .......... .......... 72% 3.46M 0s
  1150K .......... .......... .......... .......... .......... 75% 1.86M 0s
  1200K .......... .......... .......... .......... .......... 78%  895K 0s
  1250K .......... .......... .......... .......... .......... 81% 7.88M 0s
  1300K .......... .......... .......... .......... .......... 85% 1.17M 0s
  1350K .......... .......... .......... .......... .......... 88% 1.49M 0s
  1400K .......... .......... .......... .......... .......... 91% 4.92M 0s
  1450K .......... .......... .......... .......... .......... 94%  841K 0s
  1500K ....

  4400K .......... .......... .......... .......... .......... 78%  290K 1s
  4450K .......... .......... .......... .......... .......... 79% 18.4M 1s
  4500K .......... .......... .......... .......... .......... 80%  292K 1s
  4550K .......... .......... .......... .......... .......... 81% 18.0M 1s
  4600K .......... .......... .......... .......... .......... 81% 15.4M 1s
  4650K .......... .......... .......... .......... .......... 82%  299K 1s
  4700K .......... .......... .......... .......... .......... 83% 16.4M 1s
  4750K .......... .......... .......... .......... .......... 84%  289K 1s
  4800K .......... .......... .......... .......... .......... 85% 16.6M 1s
  4850K .......... .......... .......... .......... .......... 86% 17.1M 1s
  4900K .......... .......... .......... .......... .......... 87%  319K 1s
  4950K .......... .......... .......... .......... .......... 88% 16.1M 1s
  5000K .......... .......... .......... .......... .......... 88%  358K 1s
  5050K ....

...

  6300K .......... .......... .......... .......... .......... 34%  235K 12s
  6350K .......... .......... .......... .......... .......... 34% 7.37M 12s
  6400K .......... .......... .......... .......... .......... 34% 10.9M 12s
  6450K .......... .......... .......... .......... .......... 35%  285K 12s
  6500K .......... .......... .......... .......... .......... 35% 10.0M 12s
  6550K .......... .......... .......... .......... .......... 35%  298K 12s
  6600K .......... .......... .......... .......... .......... 35% 9.03M 12s
  6650K .......... .......... .......... .......... .......... 36% 11.0M 12s
  6700K .......... .......... .......... .......... .......... 36%  270K 12s
  6750K .......... .......... .......... .......... .......... 36% 12.0M 12s
  6800K .......... .......... .......... .......... .......... 37% 7.77M 12s
  6850K .......... .......... .......... .......... .......... 37%  297K 12s
  6900K .......... .......... .......... .......... .......... 37% 7.61M 12s

 11850K .......... .......... .......... .......... .......... 64% 10.7M 8s
 11900K .......... .......... .......... .......... .......... 64%  279K 8s
 11950K .......... .......... .......... .......... .......... 64% 11.4M 8s
 12000K .......... .......... .......... .......... .......... 65%  300K 8s
 12050K .......... .......... .......... .......... .......... 65% 9.38M 8s
 12100K .......... .......... .......... .......... .......... 65%  274K 8s
 12150K .......... .......... .......... .......... .......... 65% 11.0M 8s
 12200K .......... .......... .......... .......... .......... 66% 7.48M 8s
 12250K .......... .......... .......... .......... .......... 66%  296K 8s
 12300K .......... .......... .......... .......... .......... 66% 7.89M 8s
 12350K .......... .......... .......... .......... .......... 67% 11.4M 7s
 12400K .......... .......... .......... .......... .......... 67%  256K 7s
 12450K .......... .......... .......... .......... .......... 67% 9.69M 7s
 12500K ....

 17300K .......... .......... .......... .......... .......... 93%  141K 1s
 17350K .......... .......... .......... .......... .......... 94% 13.0M 1s
 17400K .......... .......... .......... .......... .......... 94% 14.1M 1s
 17450K .......... .......... .......... .......... .......... 94%  271K 1s
 17500K .......... .......... .......... .......... .......... 94% 12.0M 1s
 17550K .......... .......... .......... .......... .......... 95%  284K 1s
 17600K .......... .......... .......... .......... .......... 95% 12.5M 1s
 17650K .......... .......... .......... .......... .......... 95% 13.4M 1s
 17700K .......... .......... .......... .......... .......... 95%  290K 1s
 17750K .......... .......... .......... .......... .......... 96% 10.3M 1s
 17800K .......... .......... .......... .......... .......... 96%  272K 1s
 17850K .......... .......... .......... .......... .......... 96% 7.05M 1s
 17900K .......... .......... .......... .......... .......... 97% 10.2M 1s
 17950K ....

  3900K .......... .......... .......... .......... .......... 49%  386K 3s
  3950K .......... .......... .......... .......... .......... 49% 11.1M 3s
  4000K .......... .......... .......... .......... .......... 50%  351K 3s
  4050K .......... .......... .......... .......... .......... 50% 8.39M 3s
  4100K .......... .......... .......... .......... .......... 51%  341K 3s
  4150K .......... .......... .......... .......... .......... 52% 11.2M 3s
  4200K .......... .......... .......... .......... .......... 52%  154K 3s
  4250K .......... .......... .......... .......... .......... 53% 9.92M 3s
  4300K .......... .......... .......... .......... .......... 54%  126K 3s
  4350K .......... .......... .......... .......... .......... 54% 5.52M 3s
  4400K .......... .......... .......... .......... .......... 55% 5.96M 3s
  4450K .......... .......... .......... .......... .......... 55%  354K 3s
  4500K .......... .......... .......... .......... .......... 56% 8.42M 3s
  4550K ....

TimeoutExpired: Command 'wget -r -np -nH --cut-dirs=3 -R index.html -R index.html.tmp http://logs.ute.nsn-rdnet.net/cloud/execution/15632187/' timed out after 300 seconds

In [6]:
storage.save("syslog", "http://logs.ute.nsn-rdnet.net/cloud/execution/15632187/execution_for_preparation_2e17f041-3457-4ff4-ab71-871c73366439/test_results/syslog/")

--2022-08-24 17:01:42--  http://logs.ute.nsn-rdnet.net/cloud/execution/15632187/execution_for_preparation_2e17f041-3457-4ff4-ab71-871c73366439/test_results/syslog/
Resolving lab-proxy.krk-lab.nsn-rdnet.net (lab-proxy.krk-lab.nsn-rdnet.net)... 10.158.100.2
Connecting to lab-proxy.krk-lab.nsn-rdnet.net (lab-proxy.krk-lab.nsn-rdnet.net)|10.158.100.2|:8080... connected.
Proxy request sent, awaiting response... 200 OK
Length: 1326 (1.3K) [text/html]
Saving to: ‘execution_for_preparation_2e17f041-3457-4ff4-ab71-871c73366439/test_results/syslog/index.html.tmp’

     0K .                                                     100%  126M=0s

2022-08-24 17:01:44 (126 MB/s) - ‘execution_for_preparation_2e17f041-3457-4ff4-ab71-871c73366439/test_results/syslog/index.html.tmp’ saved [1326/1326]

Loading robots.txt; please ignore errors.
--2022-08-24 17:01:45--  http://logs.ute.nsn-rdnet.net/robots.txt
Reusing existing connection to lab-proxy.krk-lab.nsn-rdnet.net:8080.
Proxy request sent, awaiting resp

  4300K .......... .......... .......... .......... .......... 53% 4.22M 3s
  4350K .......... .......... .......... .......... .......... 53% 5.08M 3s
  4400K .......... .......... .......... .......... .......... 54% 4.64M 3s
  4450K .......... .......... .......... .......... .......... 54% 3.36M 3s
  4500K .......... .......... .......... .......... .......... 55% 9.19M 3s
  4550K .......... .......... .......... .......... .......... 56% 9.42M 3s
  4600K .......... .......... .......... .......... .......... 56% 2.77M 3s
  4650K .......... .......... .......... .......... .......... 57% 1.15M 3s
  4700K .......... .......... .......... .......... .......... 57% 6.17M 3s
  4750K .......... .......... .......... .......... .......... 58%  301K 3s
  4800K .......... .......... .......... .......... .......... 59% 6.54M 3s
  4850K .......... .......... .......... .......... .......... 59% 10.4M 3s
  4900K .......... .......... .......... .......... .......... 60%  288K 3s
  4950K ....

  1200K .......... .......... .......... .......... .......... 18% 1.03M 2s
  1250K .......... .......... .......... .......... .......... 18% 2.50M 2s
  1300K .......... .......... .......... .......... .......... 19% 1.29M 2s
  1350K .......... .......... .......... .......... .......... 20% 1.74M 2s
  1400K .......... .......... .......... .......... .......... 20% 1.90M 2s
  1450K .......... .......... .......... .......... .......... 21% 1.31M 2s
  1500K .......... .......... .......... .......... .......... 22% 1.44M 2s
  1550K .......... .......... .......... .......... .......... 23% 1.99M 2s
  1600K .......... .......... .......... .......... .......... 23% 1.52M 2s
  1650K .......... .......... .......... .......... .......... 24% 1.56M 2s
  1700K .......... .......... .......... .......... .......... 25% 2.16M 2s
  1750K .......... .......... .......... .......... .......... 25% 1.26M 2s
  1800K .......... .......... .......... .......... .......... 26% 1.86M 2s
  1850K ....

  6700K .......... .......... .......... .......... .......... 97%  297K 0s
  6750K .......... .......... .......... .......... .......... 98% 10.0M 0s
  6800K .......... .......... .......... .......... .......... 98% 10.5M 0s
  6850K .......... .......... .......... .......... .......... 99%  295K 0s
  6900K .......... .......... .......... .....                100% 12.7M=6.9s

2022-08-24 17:02:10 (1005 KB/s) - ‘execution_for_preparation_2e17f041-3457-4ff4-ab71-871c73366439/test_results/syslog/syslog_gnb_QB-0002_127.0.0.1.log’ saved [7101968/7101968]

--2022-08-24 17:02:10--  http://logs.ute.nsn-rdnet.net/cloud/execution/15632187/execution_for_preparation_2e17f041-3457-4ff4-ab71-871c73366439/test_results/syslog/syslog_gnb_RB-0001_127.0.0.1.log
Reusing existing connection to lab-proxy.krk-lab.nsn-rdnet.net:8080.
Proxy request sent, awaiting response... 200 OK
Length: 17994138 (17M) [text/plain]
Saving to: ‘execution_for_preparation_2e17f041-3457-4ff4-ab71-871c73366439/test_results/sys

  4900K .......... .......... .......... .......... .......... 28%  305K 11s
  4950K .......... .......... .......... .......... .......... 28% 13.2M 11s
  5000K .......... .......... .......... .......... .......... 28%  287K 11s
  5050K .......... .......... .......... .......... .......... 29% 10.5M 11s
  5100K .......... .......... .......... .......... .......... 29% 10.8M 11s
  5150K .......... .......... .......... .......... .......... 29%  292K 11s
  5200K .......... .......... .......... .......... .......... 29% 7.95M 11s
  5250K .......... .......... .......... .......... .......... 30%  301K 11s
  5300K .......... .......... .......... .......... .......... 30% 11.5M 11s
  5350K .......... .......... .......... .......... .......... 30% 11.5M 11s
  5400K .......... .......... .......... .......... .......... 31%  289K 11s
  5450K .......... .......... .......... .......... .......... 31% 9.89M 11s
  5500K .......... .......... .......... .......... .......... 31% 12.5M 11s

...

  6900K .......... .......... .......... .......... .......... 37%  292K 12s
  6950K .......... .......... .......... .......... .......... 37% 7.73M 12s
  7000K .......... .......... .......... .......... .......... 38%  299K 12s
  7050K .......... .......... .......... .......... .......... 38% 11.3M 12s
  7100K .......... .......... .......... .......... .......... 38% 13.0M 12s
  7150K .......... .......... .......... .......... .......... 38%  297K 12s
  7200K .......... .......... .......... .......... .......... 39% 10.7M 12s
  7250K .......... .......... .......... .......... .......... 39%  297K 12s
  7300K .......... .......... .......... .......... .......... 39% 9.21M 12s
  7350K .......... .......... .......... .......... .......... 40% 4.87M 11s
  7400K .......... .......... .......... .......... .......... 40%  301K 12s
  7450K .......... .......... .......... .......... .......... 40% 11.3M 11s
  7500K .......... .......... .......... .......... .......... 40%  292K 12s

 12450K .......... .......... .......... .......... .......... 67%  286K 7s
 12500K .......... .......... .......... .......... .......... 67% 10.1M 7s
 12550K .......... .......... .......... .......... .......... 68%  285K 7s
 12600K .......... .......... .......... .......... .......... 68% 13.8M 7s
 12650K .......... .......... .......... .......... .......... 68% 10.3M 7s
 12700K .......... .......... .......... .......... .......... 68%  286K 7s
 12750K .......... .......... .......... .......... .......... 69% 8.94M 7s
 12800K .......... .......... .......... .......... .......... 69% 15.0M 7s
 12850K .......... .......... .......... .......... .......... 69%  295K 7s
 12900K .......... .......... .......... .......... .......... 70% 15.3M 7s
 12950K .......... .......... .......... .......... .......... 70%  302K 7s
 13000K .......... .......... .......... .......... .......... 70% 13.6M 7s
 13050K .......... .......... .......... .......... .......... 70%  297K 7s
 13100K ....

 17850K .......... .......... .......... .......... .......... 96%  293K 1s
 17900K .......... .......... .......... .......... .......... 97% 9.83M 1s
 17950K .......... .......... .......... .......... .......... 97% 12.4M 1s
 18000K .......... .......... .......... .......... .......... 97%  287K 1s
 18050K .......... .......... .......... .......... .......... 97% 11.1M 0s
 18100K .......... .......... .......... .......... .......... 98%  305K 0s
 18150K .......... .......... .......... .......... .......... 98% 12.3M 0s
 18200K .......... .......... .......... .......... .......... 98% 12.9M 0s
 18250K .......... .......... .......... .......... .......... 98%  278K 0s
 18300K .......... .......... .......... .......... .......... 99% 8.93M 0s
 18350K .......... .......... .......... .......... .......... 99% 13.8M 0s
 18400K .......... .......... .......... .......... .......... 99%  273K 0s
 18450K .......... .......... .......... ..........           100% 8.92M=23s

2022-08-24

  4350K .......... .......... .......... .......... .......... 54% 12.9M 3s
  4400K .......... .......... .......... .......... .......... 55%  306K 3s
  4450K .......... .......... .......... .......... .......... 55% 13.8M 3s
  4500K .......... .......... .......... .......... .......... 56%  285K 3s
  4550K .......... .......... .......... .......... .......... 57% 8.23M 3s
  4600K .......... .......... .......... .......... .......... 57%  303K 3s
  4650K .......... .......... .......... .......... .......... 58% 10.6M 3s
  4700K .......... .......... .......... .......... .......... 59% 7.02M 3s
  4750K .......... .......... .......... .......... .......... 59%  310K 3s
  4800K .......... .......... .......... .......... .......... 60% 8.65M 3s
  4850K .......... .......... .......... .......... .......... 60% 15.4M 3s
  4900K .......... .......... .......... .......... .......... 61%  326K 3s
  4950K .......... .......... .......... .......... .......... 62% 8.63M 3s
  5000K ....

True

In [ ]:
result = storage.save("syslog", "http://logs.ute.nsn-rdnet.net/cloud/execution/15632187/execution_for_preparation_2e17f041-3457-4ff4-ab71-871c73366439/test_results/syslog/")
result

In [5]:
storage.listdir("2175")

(['execution_for_preparation_2e17f041-3457-4ff4-ab71-871c73366439'],
 ['log.txt'])

In [10]:
storage.get_accessed_time("2175")

datetime.datetime(2022, 8, 24, 13, 45, 17, tzinfo=datetime.timezone.utc)

In [11]:
storage.get_created_time("2175")

datetime.datetime(2022, 8, 24, 13, 35, 37, tzinfo=datetime.timezone.utc)

In [12]:
storage.get_modified_time("2175")

datetime.datetime(2022, 8, 24, 13, 35, 37, tzinfo=datetime.timezone.utc)

In [7]:
from urllib.parse import urlparse

path = urlparse("http://logs.ute.nsn-rdnet.net/cloud/execution/15632187/").path
path


'/cloud/execution/15632187/'

In [8]:
path[:-1]

'/cloud/execution/15632187'